# NILMAnalytics 

## Remarks:
### 1. 'rolling fourier' (is dit hetzelfde als een STFT of niet omwillen van de sprong per cycle)
__Possible improvements:__

   * Gaussian window? (zoals Gabortranform: optimum freq/tijd-resolutie)
   * wavelets (zoals Morlet) frequentieresolutie is relatief gezien constant
   
### 2. Is het noodzakelijk om elke cyclus naar een zerocrossing te zoeken?
   * Zerocrossing lijkt op het eerste zicht overbodig omdat enkel te relatieve hoek tussen spanning en stroom belangrijk is. Verder best een checken of zerocrossing wel degelijk werkt. 
   
### 3. Windowed STFT: 
   * Momenteel werkt enkel een window van 8 cycles. Andere window lengtes zorgen voor een shift van de frequenties dus waarschijnlijk is de berekening van de frequenties verkeerd
   * Bij het berekenen van de STFT moet je rekening houden dat de helft van de energie in de negatieve frequenties zit. Plus het feit dat de RMS waarde een factor $ \sqrt2 $ kleiner is
   * Berekening best aanpassen met <code>rolling()</code> van pandas

### 4. Berekeking van phi:
   * Momenteel is dit enkel de hoek van de fundamenteles!
   * Moet de negatieve component ook gebruikt worden om de hoek te bereken?

In [10]:
import pandas as pd
import matplotlib.pyplot
import plotly.graph_objs as go
import numpy as np
import scipy as sp

class ivSignal(object): 
    
    def __init__(self, i_series, v_series, fs, fs_net=50):
        self.v_series = v_series
        self.i_series = i_series
        self.fs = fs
        self.fs_net = fs_net
    
    def subsample(self, dec_factor = 2):
        pass
    
    def calculateIV_rms(self, window=None):
        if window == None:
            window = self.fs/self.fs_net
            
        self.V_series = self.v_series
        self.V_series = self.V_series.apply(np.square)
        self.V_series = self.V_series.rolling(window=window, center=True).mean()
        self.V_series = self.V_series.apply(np.sqrt)
        return None
    
    def calculateWindowedSTFT(self, start_i=0, window_cycles=8, zerocross=False):
        samples_per_cycle = self.fs/self.fs_net
        window_samples = samples_per_cycle*window_cycles      
        
        #find zerocrossing to start the STFT
        if zerocross == True:
            start_i = self._find_zerocrossing(self.v_series[start:])
        
        #calculate number of segments | the minimum unit is a cycle
        number_of_samples = len(self.v_series[start_i:])
        number_of_cycles = number_of_samples//samples_per_cycle
        output_len = number_of_cycles-window_cycles+1
        
        #ceil the number of samples to a power of 2 
        n = next_pow2(window_samples)
        
        # WARNING! Maybe necessary to search for a zerocrossing each cycle
        stack_dict = {}
        for i in range(output_len):
            df = pd.DataFrame()
            start = start_i+i*samples_per_cycle
            stop = start_i+i*samples_per_cycle+window_samples
            v_fourier = 1j*np.fft.fft(self.v_series[start:stop], n)/n # times 'j' to get the standard phasor representation
            i_fourier = 1j*np.fft.fft(self.i_series[start:stop], n)/n
            v_fourier = np.fft.fftshift(v_fourier)
            i_fourier = np.fft.fftshift(i_fourier)
            f = np.arange(n)
            f = (f-float(n)/2)*self.fs/float(n)
            df = pd.DataFrame({'Frequency (Hz)': f, 'Current (A)': i_fourier, 'Voltage (V)' : v_fourier})
            df = df.set_index('Frequency (Hz)')
            stack_dict[i] = df
        self.STFT = pd.Panel(stack_dict)
        return self.STFT
    
    def calculateFFT(self, start=0, window_in_cycles=1):
        samples_per_cycle = self.fs/self.fs_net
        window_in_samples = samples_per_cycle*cycles
        n = next_pow2(window_in_samples)
        start_i = self._find_zerocrossing(self.v_series[start:])
        start_i=0
        df = pd.DataFrame()
        if not (start_i == None):
            v_fourier = 1j*np.fft.fft(self.v_series[start_i:start_i+window_in_samples], n)/n
            i_fourier = 1j*np.fft.fft(self.i_series[start_i:start_i+window_in_samples], n)/n
            v_fourier = np.fft.fftshift(v_fourier)
            i_fourier = np.fft.fftshift(i_fourier)
            f = np.arange(n)
            f = (f-float(n)/2)*self.fs/float(n)
            df = pd.DataFrame({'Frequency (Hz)': f, 'Current (A)': i_fourier, 'Voltage (V)' : v_fourier})
            df = df.set_index('Frequency (Hz)')
        return df
    
    def calculateSTFT(self, window_in_cycles = 8, std = 1):
        pass
                
    def calculatePQ(self):
        P = {}
        Q = {}
        for index, df in self.STFT.iteritems():
            I_comps = df.loc[[-5*self.fs_net,-3*self.fs_net,-1*self.fs_net,self.fs_net,3*self.fs_net,5*self.fs_net]]['Current (A)']
            U_comps = df.loc[[-5*self.fs_net,-3*self.fs_net,-1*self.fs_net,self.fs_net,3*self.fs_net,5*self.fs_net]]['Voltage (V)']
            Phi_comps = np.angle(I_comps)-np.angle(U_comps)
            UI_comps = np.abs(U_comps*I_comps)
            P_comps = UI_comps*np.cos(Phi_comps)
            Q_comps = UI_comps*np.sin(Phi_comps)
            P[index] = P_comps.sum()
            Q[index] = Q_comps.sum()        
        P_series = pd.Series(P)
        Q_series = pd.Series(Q)
        self.PQ = pd.DataFrame({'Active Power (W)': P_series, 'Reactive Power (VAr)': Q_series})
        return self.PQ
        
    
    def calculatePhi(self):
        #WARNING! This is only the phi of the fundamental!
        series = {}
        for index, df in self.STFT.iteritems():
            #WARNING!: check if the negative component isn't necessary for calc of phi
            series[index] = np.angle(df[-self.fs_net:-self.fs_net]['Current (A)'])-np.angle(df[-self.fs_net:-self.fs_net]['Voltage (V)']);
        self.Phi = pd.Series(series, dtype='float')
        return self.Phi
    
    def _find_zerocrossing(self, series):
        for (i, y) in series.iteritems():
            if i < series.size:
                if not (np.sign(series.iloc[i]) <> np.sign(series.iloc[i+1])):
                    return i
        return None
    
    
def next_pow2(n):
    return int(np.square(np.ceil(np.sqrt(n))))
        
        